# AutoARIMA on Stock Prices

<img src="intro.png">

In [1]:
#Install needed dependencies, in cases where they are not available already
!pip install pmdarima
!pip install plotly
!pip install pandas
!pip install yfinance

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
from pmdarima.arima import AutoARIMA
import plotly.express as px
from statistics import mean
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import datetime, date, timedelta
import yfinance as yf

Choosing Stocks that have significantly lost value in the past few years

In [3]:
# # Getting the timeframe between 2015 and 2020 to download the crypto data from yahoo!
# endPeriod = date.today()
# startTradePeriod = datetime.fromisoformat("2021-01-01 00:00:00")
# startPeriod = startTradePeriod - timedelta(weeks=260)

# # Crypto currencies to analyze
# stocks = ['BTC-USD', 'ETH-USD']

# #Retrieve the data from yahoo! finance
# df = yf.download(stocks, start=startPeriod.strftime("%Y-%m-%d"), end=endPeriod.strftime("%Y-%m-%d")).dropna()

# print("Rows in DataFrame: ", df.shape[0])
# display(df)

In [4]:
# # Getting the timeframe between 2015 and 2020 to download the crypto data from yahoo!
endPeriod = date.today()
startTradePeriod = datetime.fromisoformat("2021-01-01 00:00:00")
startPeriod = startTradePeriod - timedelta(weeks=260)

# # Crypto currencies to analyze
stocks = ['BTC-USD', 'ETH-USD', 'BCH-USD', 'DOGE-USD', 'LTC-USD', 'ADA-USD', 'DASH-USD', 'XMR-USD', 'XRP-USD', 'USDT-USD']
columns_to_drop = ['Open', 'High', 'Low', 'Close', 'Volume']

end = datetime.now()
start = datetime(end.year -1, end.month, end.day)

#Retrieve data from the already downloaded yahoo csv files
df = {}
for i in range(len(stocks)):
    coin_df = pd.read_csv(stocks[i] + '.csv', encoding='utf8').drop(columns=columns_to_drop)
    coin_df.reset_index(drop=True, inplace=True)
    
    if i == 0:
        df = coin_df
    else:
        df = pd.merge(df,coin_df,on="Date",how="left")
        
df.reset_index(drop=True, inplace=True)
        

df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
df = df.drop(columns=['Date'])



#init multiindex columns
nested_columns = [['Adj Close' for _ in range(len(stocks))], stocks]
df.columns = pd.MultiIndex.from_arrays(nested_columns)
df = df.dropna()

print("Rows in DataFrame: ", df.shape[0])
display(df)

Rows in DataFrame:  1375


Adj Close                                                 \
                 BTC-USD      ETH-USD     BCH-USD  DOGE-USD     LTC-USD   
Date                                                                      
2017-10-01   4403.740234   302.337006  415.152008  0.001084   54.543301   
2017-10-02   4409.319824   297.475006  421.186005  0.001085   53.375999   
2017-10-03   4317.479980   292.463013  404.177002  0.001108   52.349602   
2017-10-04   4229.359863   292.657990  358.446991  0.001024   51.399502   
2017-10-05   4328.410156   295.863007  355.750000  0.001001   51.669498   
...                  ...          ...         ...       ...         ...   
2021-07-06  34235.195313  2324.679443  511.667114  0.234422  138.985641   
2021-07-07  33855.328125  2315.161865  508.864532  0.224126  137.371216   
2021-07-08  32877.371094  2120.026367  490.991058  0.207597  132.308502   
2021-07-09  33798.011719  2146.692383  504.062988  0.219864  134.571594   
2021-07-10  33821.960938  2113.705078  495.827362  0.216874  132.975723   

                                                                  
             ADA-USD    DASH-USD     XMR-USD   XRP-USD  USDT-USD  
Date                                                              
2017-10-01  0.024969  315.802002   93.222000  0.206532  0.999924  
2017-10-02  0.025932  308.684998   91.544197  0.202676  1.001700  
2017-10-03  0.020816  296.234985   92.961197  0.203318  1.000120  
2017-10-04  0.021931  305.457001   90.407097  0.214774  1.001250  
2017-10-05  0.021489  305.398987   91.858902  0.237016  1.000300  
...              ...         ...         ...       ...       ...  
2021-07-06  1.418053  134.897461  222.137131  0.665402  1.000090  
2021-07-07  1.403118  134.566879  214.850555  0.651575  1.000708  
2021-07-08  1.330146  126.603027  205.441284  0.623058  0.999900  
2021-07-09  1.349474  128.496262  212.746994  0.636804  0.999639  
2021-07-10  1.338711  126.896309  215.710281  0.625282  1.000138  

[1375 rows x 10 columns]

In [5]:
# Storing the dataframes in a dictionary
stock_df = {}

for col in set(df.columns.get_level_values(0)):
    
    # Assigning the information (High, Low, etc.) for each stock in the dictionary
    stock_df[col] = df[col]

# Preprocessing Data

Scale the data using a logarithmic scale.  Also rounding the log result by 2 decimal points in order to reduce any unnecessary noise.

In [6]:
# Finding the log returns
stock_df['LogReturns'] = stock_df['Adj Close'].apply(np.log).diff().dropna()

# Trying out Moving average
stock_df['MovAvg'] = stock_df['Adj Close'].rolling(10).mean().dropna()

# Logarithmic scaling of the data and rounding the result
stock_df['Log'] = stock_df['MovAvg'].apply(np.log).apply(lambda x: round(x, 2))

display(stock_df)

{'Adj Close':                  BTC-USD      ETH-USD     BCH-USD  DOGE-USD     LTC-USD  \
 Date                                                                      
 2017-10-01   4403.740234   302.337006  415.152008  0.001084   54.543301   
 2017-10-02   4409.319824   297.475006  421.186005  0.001085   53.375999   
 2017-10-03   4317.479980   292.463013  404.177002  0.001108   52.349602   
 2017-10-04   4229.359863   292.657990  358.446991  0.001024   51.399502   
 2017-10-05   4328.410156   295.863007  355.750000  0.001001   51.669498   
 ...                  ...          ...         ...       ...         ...   
 2021-07-06  34235.195313  2324.679443  511.667114  0.234422  138.985641   
 2021-07-07  33855.328125  2315.161865  508.864532  0.224126  137.371216   
 2021-07-08  32877.371094  2120.026367  490.991058  0.207597  132.308502   
 2021-07-09  33798.011719  2146.692383  504.062988  0.219864  134.571594   
 2021-07-10  33821.960938  2113.705078  495.827362  0.216874  132.975723   

In [7]:
#select trading dataframe
trading_df = df.loc[startTradePeriod <= df.index]

display(trading_df)

Adj Close                                                 \
                 BTC-USD      ETH-USD     BCH-USD  DOGE-USD     LTC-USD   
Date                                                                      
2021-01-01  29374.152344   730.367554  341.985657  0.005685  126.230347   
2021-01-02  32127.267578   774.534973  354.599945  0.010615  136.944885   
2021-01-03  32782.023438   975.507690  421.789856  0.009771  160.190582   
2021-01-04  31971.914063  1040.233032  407.936920  0.009767  154.807327   
2021-01-05  33992.429688  1100.006104  419.768127  0.009920  158.594772   
...                  ...          ...         ...       ...         ...   
2021-07-06  34235.195313  2324.679443  511.667114  0.234422  138.985641   
2021-07-07  33855.328125  2315.161865  508.864532  0.224126  137.371216   
2021-07-08  32877.371094  2120.026367  490.991058  0.207597  132.308502   
2021-07-09  33798.011719  2146.692383  504.062988  0.219864  134.571594   
2021-07-10  33821.960938  2113.705078  495.827362  0.216874  132.975723   

                                                                  
             ADA-USD    DASH-USD     XMR-USD   XRP-USD  USDT-USD  
Date                                                              
2021-01-01  0.175350   88.241707  136.045868  0.237444  1.001916  
2021-01-02  0.177423   86.969772  143.117218  0.221655  1.000600  
2021-01-03  0.204995   89.069138  138.064423  0.225848  1.000514  
2021-01-04  0.224762   88.797173  133.602768  0.236677  1.000128  
2021-01-05  0.258314   89.102776  136.825089  0.226750  1.002202  
...              ...         ...         ...       ...       ...  
2021-07-06  1.418053  134.897461  222.137131  0.665402  1.000090  
2021-07-07  1.403118  134.566879  214.850555  0.651575  1.000708  
2021-07-08  1.330146  126.603027  205.441284  0.623058  0.999900  
2021-07-09  1.349474  128.496262  212.746994  0.636804  0.999639  
2021-07-10  1.338711  126.896309  215.710281  0.625282  1.000138  

[191 rows x 10 columns]

# Visualizing the Data

In [8]:
px.line(stock_df['MovAvg'], 
        x=stock_df['MovAvg'].index, 
        y=stock_df['MovAvg'].columns,
        labels={'variable': 'Stock',
                'value': 'Price'},
        title='Moving Average')


In [9]:
px.line(stock_df['Log'], 
        x=stock_df['Log'].index, 
        y=stock_df['Log'].columns,
        labels={'variable': 'Stock',
                'value': 'Log Scale'},
        title='Log of Moving Averages')

## Optimum Parameter Search Function
Класически авторегресионен - p, 0, 0. При него спайк на PACF показва  p

Класически MA (0,0,q). Спайкът на ACF показва q

Смесени модели, без спайкове (1,0,1)

ARMA 1,0,1 - Когато и двете плавно намаляват, без спайкове


In [10]:

opt_param = AutoARIMA(start_p=0, start_q=0,
                      start_P=0, start_Q=0,
                      max_p=8, max_q=8,
                      max_P=5, max_Q=5,
                      error_action='ignore',
                      information_criterion='bic',
                      suppress_warnings=True)

for stock in tqdm(stocks):

    opt_param.fit(stock_df['Log'][stock])

    print(f'Summary for {stock}', '--'*20)
    display(opt_param.summary())

  0%|          | 0/10 [00:00<?, ?it/s]

Summary for BTC-USD ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1366
Model:               SARIMAX(2, 1, 0)   Log Likelihood                4614.957
Date:                Sun, 11 Jul 2021   AIC                          -9223.913
Time:                        12:10:33   BIC                          -9208.256
Sample:                             0   HQIC                         -9218.053
                               - 1366                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5143      0.023     22.374      0.000       0.469       0.559
ar.L2          0.3570      0.024     15.045      0.000       0.310       0.403
sigma2       6.76e-05    2.3e-06     29.414      0.000    6.31e-05    7.21e-05
===================================================================================
Ljung-Box (L1) (Q):                   0.04   Jarque-Bera (JB):                23.45
Prob(Q):                              0.84   Prob(JB):                         0.00
Heteroskedasticity (H):               0.78   Skew:                            -0.02
Prob(H) (two-sided):                  0.01   Kurtosis:                         3.64
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for ETH-USD ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1366
Model:               SARIMAX(1, 2, 0)   Log Likelihood                4414.990
Date:                Sun, 11 Jul 2021   AIC                          -8825.981
Time:                        12:10:34   BIC                          -8815.544
Sample:                             0   HQIC                         -8822.074
                               - 1366                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.3187      0.025    -12.884      0.000      -0.367      -0.270
sigma2      9.037e-05   2.44e-06     36.993      0.000    8.56e-05    9.52e-05
===================================================================================
Ljung-Box (L1) (Q):                   0.17   Jarque-Bera (JB):               257.29
Prob(Q):                              0.68   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                             0.16
Prob(H) (two-sided):                  0.02   Kurtosis:                         5.10
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

...

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1366
Model:               SARIMAX(2, 1, 0)   Log Likelihood                4379.549
Date:                Sun, 11 Jul 2021   AIC                          -8753.097
Time:                        12:11:05   BIC                          -8737.441
Sample:                             0   HQIC                         -8747.237
                               - 1366                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5605      0.022     25.929      0.000       0.518       0.603
ar.L2          0.3111      0.021     14.508      0.000       0.269       0.353
sigma2      9.556e-05   2.69e-06     35.573      0.000    9.03e-05       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.08   Jarque-Bera (JB):               181.55
Prob(Q):                              0.78   Prob(JB):                         0.00
Heteroskedasticity (H):               0.66   Skew:                             0.10
Prob(H) (two-sided):                  0.00   Kurtosis:                         4.78
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for XRP-USD ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1366
Model:               SARIMAX(3, 1, 0)   Log Likelihood                4083.397
Date:                Sun, 11 Jul 2021   AIC                          -8158.794
Time:                        12:11:11   BIC                          -8137.918
Sample:                             0   HQIC                         -8150.980
                               - 1366                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7515      0.019     38.872      0.000       0.714       0.789
ar.L2          0.2248      0.027      8.346      0.000       0.172       0.278
ar.L3         -0.0950      0.020     -4.690      0.000      -0.135      -0.055
sigma2         0.0001   2.88e-06     51.091      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):              8814.32
Prob(Q):                              0.88   Prob(JB):                         0.00
Heteroskedasticity (H):               0.77   Skew:                             1.32
Prob(H) (two-sided):                  0.00   Kurtosis:                        15.16
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for USDT-USD ----------------------------------------


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1366
Model:               SARIMAX(0, 1, 0)   Log Likelihood                6693.402
Date:                Sun, 11 Jul 2021   AIC                         -13384.805
Time:                        12:11:13   BIC                         -13379.586
Sample:                             0   HQIC                        -13382.852
                               - 1366                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2      3.221e-06   3.18e-08    101.292      0.000    3.16e-06    3.28e-06
===================================================================================
Ljung-Box (L1) (Q):                  11.31   Jarque-Bera (JB):             44662.42
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.20   Skew:                             0.00
Prob(H) (two-sided):                  0.00   Kurtosis:                        31.02
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

# Using the ARIMA Model
Using the price history from the past N days to make predictions

In [11]:
# Days in the past to train on
days_to_train = 180 

# Days in the future to predict
days_to_predict = 5

# Establishing a new DF for predictions
stock_df['Predictions'] = pd.DataFrame(index=stock_df['Log'].index,
                                       columns=stock_df['Log'].columns)

# Iterate through each stock
for stock in tqdm(stocks):
    
    # Current predicted value
    pred_val = 0
    
    # Training the model in a predetermined date range
    for day in tqdm(range(stock_df['Log'].shape[0]-trading_df.shape[0], 
                          stock_df['Log'].shape[0]-days_to_predict)):        

        # Data to use, containing a specific amount of days
        training = stock_df['Log'][stock].iloc[day-days_to_train:day+1].dropna()
        
        # Determining if the actual value crossed the predicted value
        cross = ((training[-1] >= pred_val >= training[-2]) or 
                 (training[-1] <= pred_val <= training[-2]))
        
        # Running the model when the latest training value crosses the predicted value or every other day 
        if cross or day % 2 == 0:

            # Finding the best parameters and pick invidual ARIMA method for each stock
            model    = AutoARIMA(start_p=0, start_q=0,
                                 start_P=0, start_Q=0,
                                 max_p=8, max_q=8,
                                 max_P=5, max_Q=5,
                                 error_action='ignore',
                                 information_criterion='bic',
                                 suppress_warnings=True)

            # Getting predictions for the optimum parameters by fitting to the training set            
            forecast = model.fit_predict(training,
                                         n_periods=days_to_predict)

            # Getting the last predicted value from the next N days
            stock_df['Predictions'][stock].iloc[day:day+days_to_predict] = np.exp(forecast[-1])


            # Updating the current predicted value
            pred_val = forecast[-1]


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

  0%|          | 0/186 [00:00<?, ?it/s]

...

  0%|          | 0/186 [00:00<?, ?it/s]

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.p

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.p

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.p

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.py:460: UserWarning:

Input time-series is completely constant; returning a (0, 0, 0) ARMA.

C:\Users\Krasi\anaconda3\lib\site-packages\pmdarima\arima\auto.p

# Predictions vs Actual Values

In [12]:
# Shift ahead by one position to compare the actual values to the predictions
pred_df = stock_df['Predictions'].shift(1).astype(float).dropna()

pred_df

BTC-USD      ETH-USD     BCH-USD  DOGE-USD     LTC-USD  \
Date                                                                      
2021-01-03  31086.948001   753.694537  361.849881  0.007830  144.915428   
2021-01-04  31086.948001   753.694537  361.849881  0.007830  144.915428   
2021-01-05  33789.681982   993.032203  394.432373  0.009630  150.169281   
2021-01-06  33789.681982   993.032203  394.432373  0.009630  150.169281   
2021-01-07  36168.368327  1148.657067  412.156825  0.011143  165.670355   
...                  ...          ...         ...       ...         ...   
2021-07-06  34200.652438  2368.471288  522.345865  0.242378  142.557092   
2021-07-07  34200.652438  2368.471288  522.345865  0.242378  142.557092   
2021-07-08  34200.652438  2368.471288  522.345865  0.242378  142.557092   
2021-07-09  34200.652438  2368.471288  522.345865  0.242378  142.557092   
2021-07-10  34200.652438  2368.471288  522.345865  0.242378  142.557092   

             ADA-USD    DASH-USD     XMR-USD   XRP-USD  USDT-USD  
Date                                                              
2021-01-03  0.180948   97.729500  152.192889  0.249983       1.0  
2021-01-04  0.180948   97.729500  152.192889  0.215140       1.0  
2021-01-05  0.207953   95.593108  143.393585  0.206466       1.0  
2021-01-06  0.207953   95.593108  143.393585  0.206466       1.0  
2021-01-07  0.313313   88.497360  139.108443  0.221816       1.0  
...              ...         ...         ...       ...       ...  
2021-07-06  1.433329  142.593796  216.956286  0.663650       1.0  
2021-07-07  1.433329  142.593796  216.956286  0.663650       1.0  
2021-07-08  1.433329  142.593796  216.956286  0.663650       1.0  
2021-07-09  1.433329  142.593796  216.956286  0.663650       1.0  
2021-07-10  1.433329  142.593796  216.956286  0.663650       1.0  

[189 rows x 10 columns]

## Plotting the Predictions
Comparing the actual values with the predictions

In [13]:
for stock in stocks:
    
    fig = go.Figure()
    
    # Plotting the actual values
    fig.add_trace(go.Scatter(x=pred_df.index,
                             y=stock_df['MovAvg'][stock].loc[pred_df.index],
                             name='Actual Moving Average',
                             mode='lines'))
    
    # Plotting the predicted values
    fig.add_trace(go.Scatter(x=pred_df.index,
                             y=pred_df[stock],
                             name='Predicted Moving Average',
                             mode='lines'))
    
    # Setting the labels
    fig.update_layout(title=f'Predicting the Moving Average for the Next {days_to_predict} days for {stock}',
                      xaxis_title='Date',
                      yaxis_title='Prices')
    
    fig.show()

## Evaluation Metric

In [14]:
for stock in stocks:
    
    # Finding the root mean squared error
    rmse = mean_squared_error(stock_df['MovAvg'][stock].loc[pred_df.index],
                              pred_df[stock],
                              squared=False)

    print(f"On average, the model is off by {rmse} for {stock}\n")

On average, the model is off by 2495.662619367728 for BTC-USD

On average, the model is off by 151.65715895704616 for ETH-USD

On average, the model is off by 59.10482597454226 for BCH-USD

On average, the model is off by 0.03284300294123304 for DOGE-USD

On average, the model is off by 13.450892681256132 for LTC-USD

On average, the model is off by 0.0920393646016401 for ADA-USD

On average, the model is off by 23.231067248717018 for DASH-USD

On average, the model is off by 15.180939710528897 for XMR-USD

On average, the model is off by 0.0848508205253245 for XRP-USD

On average, the model is off by 0.0007976745679800207 for USDT-USD



# Trading Signal
Turning the model into a Trading Signal

In [15]:
def get_positions(difference, thres=3, short=True):
    """
    Compares the percentage difference between actual values and the respective predictions.
    
    Returns the decision or positions to long or short based on the difference.
    
    Optional: shorting in addition to buying
    """
    
    if difference > thres/100:
        
        return 1
    
    
    elif short and difference < -thres/100:
        
        return -1
    
    
    else:
        
        return 0

### Trading DF
If the model is executed in the late hours of the day and new position is established on the next day's opening, then only a single shifting on time axis is appropriate. In the case that  a position is established on the next day, near the closing time , then the index needs to be shifted ahead by 2 positions, because the newly established position missed any gains or losses that day.  These are due to the fact that gains or losses in the day are determined when a trade is entered.

(This can also determine how long the predicted forecast remains valid.)

In [16]:
# Creating a DF for trading the model
trade_df = {}

# Generating the percentage difference between the  the actual values and predictions
trade_df['PercentDiff'] = (stock_df['Predictions'].dropna() / 
                           stock_df['MovAvg'].loc[stock_df['Predictions'].dropna().index]) - 1

# Getting positions
trade_df['Positions'] = trade_df['PercentDiff'].applymap(lambda x: get_positions(x, 
                                                                                 thres=1, 
                                                                                 short=True) / len(stocks))

# Preventing lookahead bias by shifting the positions
trade_df['Positions'] = trade_df['Positions'].shift(2).dropna()

# Getting Log Returns
trade_df['LogReturns'] = stock_df['LogReturns'].loc[trade_df['Positions'].index]                                    
    
display(trade_df['PercentDiff'].tail(20))
display(trade_df['Positions'].tail(20))

BTC-USD   ETH-USD   BCH-USD  DOGE-USD   LTC-USD   ADA-USD  \
Date                                                                     
2021-06-20  0.004956 -0.079821 -0.029256 -0.047168 -0.042406 -0.080565   
2021-06-21 -0.063356 -0.088935  -0.06636  -0.13492 -0.082071 -0.071196   
2021-06-22 -0.055588 -0.069069 -0.043751 -0.097924 -0.057197 -0.049902   
2021-06-23  -0.04582  -0.09487 -0.102538 -0.130787 -0.104241 -0.131983   
2021-06-24 -0.030948 -0.072004  -0.07925 -0.108218 -0.081031 -0.118144   
2021-06-25 -0.092019 -0.137017 -0.109781 -0.086164 -0.172753 -0.119671   
2021-06-26 -0.075641 -0.114254 -0.086072 -0.063297 -0.149095 -0.104147   
2021-06-27  -0.06374 -0.106764 -0.081409 -0.074635 -0.127097 -0.063799   
2021-06-28 -0.059979 -0.099926 -0.069564 -0.060738 -0.114711 -0.057234   
2021-06-29  0.020848 -0.013872 -0.036619 -0.032545 -0.047963  -0.04202   
2021-06-30  0.022843  -0.01527  -0.03034 -0.021729 -0.040045 -0.038614   
2021-07-01  0.015736  0.084404  0.020498  0.045393  0.020804  0.034691   
2021-07-02  0.011575  0.069765  0.009521  0.022778  0.007668  0.015871   
2021-07-03  0.037212  0.050045  0.039958   0.01834  0.064924  0.061029   
2021-07-04  0.035311  0.033339   0.03153  0.024966  0.056821  0.053012   
2021-07-05 -0.003914  0.110229  0.041308 -0.029479  0.031531  0.049118   
2021-07-06 -0.009823  0.085031  0.030443 -0.025435  0.022411  0.036403   
2021-07-07  -0.00754  0.068569  0.023983 -0.009373  0.018741  0.031339   
2021-07-08 -0.003036  0.066627   0.02593  0.010981  0.022439  0.030938   
2021-07-09  0.003016  0.067303  0.029612  0.029403   0.02918  0.032347   

            DASH-USD   XMR-USD   XRP-USD  USDT-USD  
Date                                                
2021-06-20 -0.008924 -0.004371  -0.05944 -0.000722  
2021-06-21 -0.097475 -0.025868 -0.102698  -0.00068  
2021-06-22 -0.071604 -0.010072 -0.069739 -0.000667  
2021-06-23 -0.097258 -0.077599 -0.133101 -0.000768  
2021-06-24  -0.07082 -0.060784 -0.107057  -0.00072  
2021-06-25 -0.186901 -0.115471 -0.130656 -0.000667  
2021-06-26 -0.163292 -0.092796 -0.103223 -0.000786  
2021-06-27 -0.130592 -0.097428  -0.10953 -0.000776  
2021-06-28 -0.115207 -0.075937 -0.088483 -0.000594  
2021-06-29  0.027851 -0.085037 -0.050415 -0.000451  
2021-06-30  0.034576 -0.066635 -0.039999 -0.000385  
2021-07-01  0.047835 -0.023417  0.052588 -0.000399  
2021-07-02  0.034852 -0.027943  0.034622 -0.000302  
2021-07-03  0.068604   0.01461 -0.003018 -0.000223  
2021-07-04  0.061258  0.017118 -0.006477 -0.000158  
2021-07-05  0.052939  0.017233 -0.002671 -0.000241  
2021-07-06  0.044825  0.008083 -0.010105 -0.000128  
2021-07-07  0.039065  0.006375 -0.010791 -0.000135  
2021-07-08  0.043003  0.009657 -0.007731 -0.000132  
2021-07-09  0.051798  0.011728  0.001995 -0.000094

BTC-USD  ETH-USD  BCH-USD  DOGE-USD  LTC-USD  ADA-USD  DASH-USD  \
Date                                                                          
2021-06-20      0.1     -0.1     -0.1      -0.1      0.1     -0.1      -0.1   
2021-06-21      0.0     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-22      0.0     -0.1     -0.1      -0.1     -0.1     -0.1       0.0   
2021-06-23     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-24     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-25     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-26     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-27     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-28     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-29     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-06-30     -0.1     -0.1     -0.1      -0.1     -0.1     -0.1      -0.1   
2021-07-01      0.1     -0.1     -0.1      -0.1     -0.1     -0.1       0.1   
2021-07-02      0.1     -0.1     -0.1      -0.1     -0.1     -0.1       0.1   
2021-07-03      0.1      0.1      0.1       0.1      0.1      0.1       0.1   
2021-07-04      0.1      0.1      0.0       0.1      0.0      0.1       0.1   
2021-07-05      0.1      0.1      0.1       0.1      0.1      0.1       0.1   
2021-07-06      0.1      0.1      0.1       0.1      0.1      0.1       0.1   
2021-07-07      0.0      0.1      0.1      -0.1      0.1      0.1       0.1   
2021-07-08      0.0      0.1      0.1      -0.1      0.1      0.1       0.1   
2021-07-09      0.0      0.1      0.1       0.0      0.1      0.1       0.1   

            XMR-USD  XRP-USD  USDT-USD  
Date                                    
2021-06-20      0.1     -0.1       0.0  
2021-06-21      0.0     -0.1       0.0  
2021-06-22      0.0     -0.1       0.0  
2021-06-23     -0.1     -0.1       0.0  
2021-06-24     -0.1     -0.1       0.0  
2021-06-25     -0.1     -0.1       0.0  
2021-06-26     -0.1     -0.1       0.0  
2021-06-27     -0.1     -0.1       0.0  
2021-06-28     -0.1     -0.1       0.0  
2021-06-29     -0.1     -0.1       0.0  
2021-06-30     -0.1     -0.1       0.0  
2021-07-01     -0.1     -0.1       0.0  
2021-07-02     -0.1     -0.1       0.0  
2021-07-03     -0.1      0.1       0.0  
2021-07-04     -0.1      0.1       0.0  
2021-07-05      0.1      0.0       0.0  
2021-07-06      0.1      0.0       0.0  
2021-07-07      0.1      0.0       0.0  
2021-07-08      0.0     -0.1       0.0  
2021-07-09      0.0     -0.1       0.0

## Plotting the Positions

In [17]:
# Getting the number of positions
pos = trade_df['Positions'].apply(pd.value_counts)

# Plotting total positions
fig = px.bar(pos, 
             x=pos.index, 
             y=pos.columns,
             title='Total Positions',
             labels={'variable':'Stocks',
                      'value':'Count of Positions',
                      'index':'Type of Position'})

fig.show()


# Calculating and Plotting the Potential Returns

## Returns on individual coins

In [18]:
# Calculating Returns by multiplying the positions by the log returns
returns = trade_df['Positions'] * trade_df['LogReturns']

# Calculating the performance as we take the cumulative sum of the returns and transform the values back to normal
performance = returns.cumsum().apply(np.exp)

# Plotting the performance per stock
px.line(performance,
        x=performance.index,
        y=performance.columns,
        title='Returns Per Coin Using ARIMA Forecast',
        labels={'variable':'Coins',
                'value':'Returns'})

## Returns on the Overall Portfolio

In [19]:
# Returns for the portfolio
returns = (trade_df['Positions'] * trade_df['LogReturns']).sum(axis=1)

display(returns)

Date
2021-01-04    0.005236
2021-01-05    0.034017
2021-01-06    0.042296
2021-01-07   -0.036524
2021-01-08    0.010513
                ...   
2021-07-05   -0.036772
2021-07-06    0.016179
2021-07-07   -0.002276
2021-07-08   -0.015441
2021-07-09    0.006318
Length: 187, dtype: float64

In [20]:
# Returns for the portfolio
returns = (trade_df['Positions'] * trade_df['LogReturns']).sum(axis=1)

#gold_df = yf.download('GC=F', start=returns.index[0]).iloc[0:returns.size]

gold_df = pd.read_csv('GC=F.csv', encoding='utf8').drop(columns=columns_to_drop)
gold_df.set_index(pd.DatetimeIndex(gold_df['Date']), inplace=True)
gold_df = gold_df.drop(columns=['Date'])

gold_df = gold_df.loc[returns.index[0] < gold_df.index ].iloc[0:returns.size]
gold_df = gold_df['Adj Close'].apply(np.log).diff().dropna().cumsum().apply(np.exp)

# Calculating the performance as we take the cumulative sum of the returns and transform the values back to normal
performance = returns.cumsum().apply(np.exp)

# Plotting the comparison between Gold future returns and ARIMA crypto returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=gold_df.index,
                         y=gold_df,
                         name='Gold Returns',
                         mode='lines'))

fig.add_trace(go.Scatter(x=performance.index,
                         y=performance.values,
                         name='Portfolio Returns',
                         mode='lines'))

fig.update_layout(title='Gold vs ARIMA Crypto Overall Portfolio Returns',
                  xaxis_title='Date',
                  yaxis_title='Returns')

fig.show()